# Reddit Scraping 

In [1]:
# Packages 
import praw
import pandas as pd
import math

In [2]:
# Accessing API 
reddit = praw.Reddit(client_id="zs_IJyaVCIFdTJOLOqgHVQ",#my client id
                     client_secret="6HOthHWpREuIemp_Y-SOfFbZGWH0eA",  #your client secret
                     user_agent="my user agent", #user agent name
                     username = "test_9876",     # your reddit username
                     password = "Pileaplante25")     # your reddit password

In [4]:
# Subreddits to scrape from 
sub = ['Denmark', 'buzzfeedbot', 'newsdk', 'dkpol', 'denmark2', 
        'scandinavia', 'dankmark', 'InfluencergossipDK', 'Aarhus']

Code found on the following website: https://medium.com/swlh/scraping-reddit-using-python-57e61e322486
It scrapes the posts on the specified subreddits that contains one of the keywords (specified below). It also scrapes all comments on those posts. 

It saves each as a CSV continuously. Therefore, it is needed to specify where they should be saved in the bottom of the next chunk. 

In [ ]:
for s in sub:
    subreddit = reddit.subreddit(s)   # Chosing the subreddit

# NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS

    # uncomments the keywords that you want to search for
    query = ['Karen Melchior', 'Sofie Carsten Nielsen', 'Astrid Krag',
              'Christel Schaldemose', 'Sophie Løhde', 'Ellen Trane Nørby',
              'Pernille Skipper', 'Liselott Blixt', 'Pia Kjærsgaard',
              'Pia Olsen Dyhr', 'Margrete Auken', 'Mette Abildgaard', 
              'Pernille Weiss', 'Pernille Vermund']
    
    # query = ['Morten Helveg', 'Stinus Lindgreen', 'Benny Engelbrect', 
    #           'Magnus Heunicke', 'Jan E. Jørgensen', 'Morten Løkkegård',
    #           'Jakob Elleman-Jensen', 'Nikolaj Villumsen', 'Rune Lund',
    #           'Morten Messerchmidt', 'Rasmus Nordqvist', 'Rasmus Jarlov',
    #           'Søren Pape']


    for item in query:
        post_dict = {
            "title" : [],
            "score" : [],
            "id" : [],
            "url" : [],
            "comms_num": [],
            "created" : [],
            "body" : [],
            "ups" : [],
            "downs" : [],
            "upvote_ratio" : [],
            "search_keyword" : []
        }
        comments_dict = {
            "comment_id" : [],
            "comment_parent_id" : [],
            "comment_body" : [],
            "comment_link_id" : [],
            "score" : [],
            "search_keyword" : []
        }
        for idx, submission in enumerate(subreddit.search(query,sort = "top")):
            post_dict["title"].append(submission.title)
            post_dict["score"].append(submission.score)
            post_dict["id"].append(submission.id)
            post_dict["url"].append(submission.url)
            post_dict["comms_num"].append(submission.num_comments)
            post_dict["created"].append(submission.created)
            post_dict["body"].append(submission.selftext)
            post_dict["ups"].append(submission.ups)
            post_dict["downs"].append(submission.downs)
            post_dict["upvote_ratio"].append(submission.upvote_ratio)

            post_dict["ups"][idx] = math.ceil(post_dict["ups"][idx] * post_dict["upvote_ratio"][idx])
            post_dict["downs"][idx] = math.ceil(post_dict["score"][idx] - post_dict["ups"][idx])

            post_dict["search_keyword"].append(query)
            
            ##### Acessing comments on the post
            submission.comments.replace_more(limit = 1) 
            for comment in submission.comments.list():
                comments_dict["comment_id"].append(comment.id)
                comments_dict["comment_parent_id"].append(comment.parent_id)
                comments_dict["comment_body"].append(comment.body)
                comments_dict["comment_link_id"].append(comment.link_id)
                comments_dict["score"].append(comment.score)
                comments_dict["search_keyword"].append(query)
        
        post_comments = pd.DataFrame(comments_dict)
        post_comments.to_csv("C:/Users/stine/OneDrive/melchior_job/nye_melchior/" + "comments_" + s  + "_" + item +"subreddit.csv")
        
        post_data = pd.DataFrame(post_dict)
        post_data.to_csv("C:/Users/stine/OneDrive/melchior_job/nye_melchior/" + "posts_" + s + "_" + item +"subreddit.csv")



## Join CSVs 

Joins the CSVs according to specified beginning of the files. E.g. joining all CSVs for the female politicians. 

In [ ]:
import os
import glob
import pandas as pd
os.chdir("C:/Users/stine/OneDrive/melchior_job/female_pol_data")


In [ ]:
#find all files in directory ending with ".csv"
file_start = 'posts_'
all_filenames = [i for i in glob.glob(f"{file_start}*")]
print(f"These are all of the filenames starting with posts_ {all_filenames}.")

#combine all cvs files
combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
print(combined_csv_data.shape)

combined_csv_data.drop('Unnamed: 0', inplace=True, axis=1)

combined_csv_data.to_csv("posts_all_female_updated.csv")